EXPORTING DATASET TO AWS DYNAMODB

In [20]:
import pandas as pd
import boto3

In [ ]:
# DynamoDB table

# Dataset definition
original_dataset = pd.read_csv('data/hotel_original.csv')

# VAR AWS Region
region_name = 'us-east-1'

# VAR AWS DynamoDB Table
dynamodb_table_name = 'hotel_reservation_dataset_original'  

# Send to DynamoDb
dynamodb = boto3.resource('dynamodb', region_name)

items = []
for index, row in original_dataset.iterrows():
    item = {
        'Booking_ID': {'S': (row['Booking_ID'])},
        'no_of_adults': {'N': str(row['no_of_adults'])},
        'no_of_children': {'N': str(row['no_of_children'])},
        'no_of_weekend_nights': {'N': str(row['no_of_weekend_nights'])},
        'no_of_week_nights': {'N': str(row['no_of_week_nights'])},
        'type_of_meal_plan': {'S': row['type_of_meal_plan']},
        'required_car_parking_space': {'N': str(row['required_car_parking_space'])},
        'room_type_reserved': {'S': row['room_type_reserved']},
        'lead_time': {'N': str(row['lead_time'])},
        'arrival_year': {'N': str(row['arrival_year'])},
        'arrival_month': {'N': str(row['arrival_month'])},
        'arrival_date': {'N': str(row['arrival_date'])},
        'market_segment_type': {'S': row['market_segment_type']},
        'repeated_guest': {'N': str(row['repeated_guest'])},
        'no_of_previous_cancellations': {'N': str(row['no_of_previous_cancellations'])},
        'no_of_previous_bookings_not_canceled': {'N': str(row['no_of_previous_bookings_not_canceled'])},
        'avg_price_per_room': {'N': str(row['avg_price_per_room'])},
        'no_of_special_requests': {'N': str(row['no_of_special_requests'])},
        'booking_status': {'S': row['booking_status']}
    }
    items.append(item)

dynamodb = boto3.client('dynamodb', region_name)
for item in items:
    response = dynamodb.put_item(
        TableName=dynamodb_table_name,
        Item=item
    )

IMPORTING DATASET FROM DYNAMODB

In [ ]:
# VAR AWS Region
region_name = 'us-east-1'

# VAR AWS DynamoDB Table Original Dataset
dynamodb_table_name = 'hotel_reservation_dataset_original' 

# Table
dynamodb = boto3.resource('dynamodb', region_name)
table = dynamodb.Table(dynamodb_table_name)

all_items = []
start_key = None

while True:
    #  Pagination
    if start_key:
        response = table.scan(ExclusiveStartKey=start_key)
    else:
        response = table.scan()

    # Add imported items to list
    items_import = response['Items']
    all_items.extend(items_import)

    #  Check if pagination over
    if 'LastEvaluatedKey' in response:
        start_key = response['LastEvaluatedKey']
    else:
        break
    
df = pd.DataFrame(all_items)
df.shape

CORRRECTING DATA TYPE

In [ ]:
# Only in case import from DynamoDB comes with JSON specifying all types as string

string_to_int = ["no_of_previous_bookings_not_canceled", "no_of_children", "repeated_guest", "no_of_special_requests", "no_of_adults", "arrival_year", "no_of_week_nights", "required_car_parking_space", "arrival_date", "no_of_weekend_nights", "arrival_month", "lead_time", "no_of_previous_cancellations"]

for column in string_to_int:
    df[column] = df[column].astype(int)
    
df["avg_price_per_room"] = df["avg_price_per_room"].astype(float)

print(df.dtypes)

GET ORIGINAL DATASET INFOS

In [30]:
# Original Dataset
original_dataset = pd.read_csv('../data/hotel_original.csv')

# Shows Dataset shape
rows, columns = original_dataset.shape
print(f"DataFrame: {rows} ROWS | {columns} COLUMNS")

# Shows data examples
original_dataset.head()

DataFrame: 36275 ROWS | 19 COLUMNS


,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [ ]:
# Shows data type
print(original_dataset.dtypes)

# Check null data
original_dataset.isnull().sum()

In [31]:
# Describe data
original_dataset.describe()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
count,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000
mean,1.844962,0.105279,0.810724,2.204300,0.030986,85.232557,2017.820427,7.423653,15.596995,0.025637,0.023349,0.153411,103.423539,0.619655
std,0.518715,0.402648,0.870644,1.410905,0.173281,85.930817,0.383836,3.069894,8.740447,0.158053,0.368331,1.754171,35.089424,0.786236
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2017.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,0.000000,17.000000,2018.000000,5.000000,8.000000,0.000000,0.000000,0.000000,80.300000,0.000000
50%,2.000000,0.000000,1.000000,2.000000,0.000000,57.000000,2018.000000,8.000000,16.000000,0.000000,0.000000,0.000000,99.450000,0.000000
75%,2.000000,0.000000,2.000000,3.000000,0.000000,126.000000,2018.000000,10.000000,23.000000,0.000000,0.000000,0.000000,120.000000,1.000000
max,4.000000,10.000000,7.000000,17.000000,1.000000,443.000000,2018.000000,12.000000,31.000000,1.000000,13.000000,58.000000,540.000000,5.000000
